In [14]:
import torch
import torch.nn as nn

class LearnablePerceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 1, bias=True)
    
    def forward(self, x):
        x = self.fc1(x)
        sigmoid = nn.Sigmoid()
        return sigmoid(x)


In [15]:
net = LearnablePerceptron()

In [16]:
# 1 はバイアス
X = torch.Tensor([[1, 170, 100], [1, 180, 55], [1, 160, 70], [1, 170, 70], [1, 145, 60]])
y = torch.Tensor([1, 0, 1, 0, 1])   # 1. fat

def normalize(X):  # min-max-normalize
    x_max = X.max()
    x_min = X.min()
    return (X - x_min + 1.0) / (x_max - x_min + 1.0)

print(X[:, 1])
print(X[:, 2])
normalized_h = normalize(X[:, 1])
normalized_w = normalize(X[:, 2])
X_normalized = torch.Tensor([[i.item(), j.item(), k.item()] for i,j,k in zip(X[:, 0], normalized_h, normalized_w)])


tensor([170., 180., 160., 170., 145.])
tensor([100.,  55.,  70.,  70.,  60.])


In [17]:
# bias はクラスに持たせることにしたので、削除する
X_normalized = X_normalized[:, 1:]

In [18]:
net(X_normalized)

tensor([[0.5287],
        [0.6017],
        [0.5251],
        [0.5568],
        [0.4867]], grad_fn=<SigmoidBackward0>)

In [19]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=0.5)

In [20]:
epochs = 100

In [21]:
losses = []
loss_fn = nn.BCELoss()

for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = net(X_normalized)
    loss = loss_fn(y_pred, y.view_as(y_pred))
    loss.backward()
    optimizer.step()
    losses.append(loss)


In [22]:
joshua = torch.Tensor([[1, 180, 55]])

# １つのデータだと、正規化したときに [1,1] になってしまうので、以下のようにする
# min-max は train のデータを使うと思ってもいいかも
X_new = torch.cat([X, joshua.reshape(1,3)])

def normalize_array(X):
    normalized_h = normalize(X[:, 1])
    normalized_w = normalize(X[:, 2])
    X_normalized = torch.Tensor([[i.item(), j.item(), k.item()] for i,j,k in zip(X[:, 0], normalized_h, normalized_w)])
    return X_normalized

X_new_normalized = normalize_array(X_new)

In [25]:
print(X_new_normalized)
joshua = X_new_normalized[-1, :]
joshua

tensor([[1.0000, 0.7222, 1.0000],
        [1.0000, 1.0000, 0.0217],
        [1.0000, 0.4444, 0.3478],
        [1.0000, 0.7222, 0.3478],
        [1.0000, 0.0278, 0.1304],
        [1.0000, 1.0000, 0.0217]])


tensor([1.0000, 1.0000, 0.0217])

In [24]:
'fat' if net(joshua[1:]) >= 0.5 else 'not fat'

'not fat'